<a href="https://colab.research.google.com/github/keglvir/ml_projekt_2025/blob/main/tiled_diffusion_self_tiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Tiled Diffusion**: Self-Tiling
Use the [Tiled Diffusion](https://github.com/madaror/tiled-diffusion) method to create seamless self-tiling textures from your own image using diffusion models

- **Self-tiling**: The image will wrap around itself.
- Stable Diffusion + latent tiling constraints.
- Works best for textures, environments, art, etc.

**Upload your image and run all cells** to get a tileable result.

In [ ]:
!pip install numpy==1.26.4
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers==0.20.2
!pip install transformers==4.31.0
!pip install xformers==0.0.20 timm==0.9.12
!pip install huggingface_hub==0.16.4 accelerate==0.21.0
!pip install pyvips opencv-python matplotlib
!pip uninstall -y Pillow && pip install --no-cache-dir Pillow==9.5.0
!pip uninstall -y jax jaxlib flax

In [ ]:
# Clone and move to the directory
!git clone https://github.com/madaror/tiled-diffusion.git
%cd tiled-diffusion

fatal: destination path 'tiled-diffusion' already exists and is not an empty directory.
/content/tiled-diffusion


In [ ]:
# Run self-tiling using your image
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from latent_class import LatentClass
from model import SDLatentTiling

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

input_image = Image.open("../input.jpg").convert("RGB")

prompt = "texture pattern"
negative_prompt = "blurry, deformed, poorly detailed"

lat = LatentClass(
    prompt=prompt,
    negative_prompt=negative_prompt,
    side_id=[1, 1, 2, 2],
    side_dir=['cw', 'ccw', 'cw', 'ccw'],
    source_image=input_image
)

model = SDLatentTiling()
latents_arr = [lat]

new_latents_arr = model(
    latents_arr=latents_arr,
    negative_prompt=negative_prompt,
    max_width=32,
    device=device
)

result_image = new_latents_arr[0].image

# Convert the float32 numpy array to uint8 and scale by 255
# Ensure the data is within the valid range [0, 1] before scaling
result_image_uint8 = (np.clip(result_image, 0, 1) * 255).astype(np.uint8)

Image.fromarray(result_image_uint8).save("self_tile_result.png")
plt.imshow(result_image_uint8)
plt.axis('off')
plt.title("Self-tiled Image")
plt.show()

Using device: cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 102.12 MiB is free. Process 190282 has 14.64 GiB memory in use. Of the allocated memory 14.40 GiB is allocated by PyTorch, and 117.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)